In [1]:
from helpers.data_loading import load_folder, load_sections

In [2]:
data_path = '../data/roze/'
data = load_folder(data_path)
sections = load_sections(data_path + 'sections.txt')

In [3]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import numpy as np

In [4]:
from helpers.tempogram_utils import subtract_mean_clean

In [5]:
ref_tempo = data.tempogram_cyclic.parameters['ref_tempo']
tempogram_clean_data = subtract_mean_clean(np.abs(data.tempogram.data))

In [6]:
layout = go.Layout(title='Tempogram')

fig = go.Figure(data=[], layout=layout)
fig.add_heatmap(z=tempogram_clean_data,  x=data.t.data, y=data.bpm.data, colorscale='Viridis', name='Tempogram')
fig.add_scatter(x=data.t_smooth.data, y=data.tempo_curve.data * ref_tempo, name='Tempo ref')
fig.add_scatter(x=data.t_smooth.data, y=data.tempo_curve.data * ref_tempo * 2, name='Tempo 2x')
fig.add_scatter(x=data.t_smooth.data, y=data.tempo_curve.data * ref_tempo * 4, name='Tempo 4x')
plot(fig)

'file://D:\\Developent\\CPP\\tempogram_tempo_estimation\\visualization_py\\temp-plot.html'

In [7]:
tempogram_cyclic_clean_data = subtract_mean_clean(np.abs(data.tempogram_cyclic.data))

In [8]:
layout2 = go.Layout(title='Tempogram Cyclic')

fig2 = go.Figure(data=[], layout=layout2)
fig2.add_heatmap(z=tempogram_cyclic_clean_data,  x=data.t.data, y=data.ct_y_axis.data, colorscale='Viridis', name='Tempogram')
fig2.add_scatter(x=data.t_smooth.data, y=data.tempo_curve.data, name='Tempo')
plot(fig2)

'file://D:\\Developent\\CPP\\tempogram_tempo_estimation\\visualization_py\\temp-plot.html'

In [9]:
smooth_length = data.smooth_tempogram.parameters['smooth_length']

In [10]:
start = data.t.data[0]
end = data.t.data[-1]

layout3 = go.Layout(title=f'Tempogram Smooth {smooth_length}', xaxis=dict(range=[start, end], autorange=False))

fig3 = go.Figure(data=[], layout=layout3)
fig3.add_heatmap(z=data.smooth_tempogram.data,  x=data.t_smooth.data, y=data.ct_y_axis.data, colorscale='Viridis', name='Tempogram')
fig3.add_scatter(x=data.t_smooth.data, y=data.tempo_curve.data, name='Tempo')
plot(fig3)

'file://D:\\Developent\\CPP\\tempogram_tempo_estimation\\visualization_py\\temp-plot.html'

In [11]:
def times_from_feature_rate(feature_rate, start, length):
    return [i / feature_rate for i in range(start, start + length)]

In [12]:
t_nc = times_from_feature_rate(data.novelty_curve.parameters['feature_rate'], 0, len(data.novelty_curve.data))

In [13]:
layout4 = go.Layout(title='Novelty curve')

fig4 = go.Figure(data=[], layout=layout4)
fig4.add_scatter(x=t_nc, y=data.novelty_curve.data, name='Novelty Curve')
plot(fig4)

'file://D:\\Developent\\CPP\\tempogram_tempo_estimation\\visualization_py\\temp-plot.html'

In [14]:
from helpers.signal_utils import pulse_for_section

In [15]:
feature_rate = data.novelty_curve.parameters['feature_rate']
pulses = []
multiples = [4, 2, 1]

for multiple in multiples:
    pulse = np.zeros_like(data.novelty_curve.data)
    for section in sections:
        cosine, _ = pulse_for_section(section, feature_rate, multiple=multiple)
        start = int(section.start * feature_rate)
        pulse[start:start + len(cosine)] = cosine
    pulses.append(pulse)
    
mag = max(data.novelty_curve.data) * 0.33

In [17]:
layout5 = go.Layout(title='Cosine')

fig5 = go.Figure(data=[], layout=layout5)
fig5.add_scatter(x=t_nc, y=data.novelty_curve.data, name='Novelty Curve')

for i in range(len(multiples)):
    fig5.add_scatter(x=t_nc, y=(pulses[i] * mag * (pulses[i] > 0)), name=f'1/{multiples[i] * 4} notes') 
    
plot(fig5)

'file://D:\\Developent\\CPP\\tempogram_tempo_estimation\\visualization_py\\temp-plot.html'